In [1]:
import numpy as np
import tensorflow as tf

In [2]:
np.random.seed(0)

W1_val = np.random.rand(4, 3)
b1_val = np.random.rand(4, 1)
W2_val = np.random.rand(1, 4)
b2_val = np.random.rand(1, 1)
x_val = np.random.rand(3, 1)

W1 = tf.constant(W1_val)
b1 = tf.constant(b1_val)
W2 = tf.constant(W2_val)
b2 = tf.constant(b2_val)

In [3]:
x = tf.placeholder(shape=(3,1), dtype=np.float64)

In [4]:
z1 = (W1@x+b1)
a1 = z1**2

z2 = W2@a1 + b2
a2 = z2**2

In [5]:
sess = tf.Session()

In [6]:
var = x
grads = tf.gradients(a2**2, var)
hess = tf.hessians(a2**2, var)
sess.run([grads, hess], feed_dict={x:x_val})

[[array([[ 6962.19332554],
         [ 9821.4284842 ],
         [10552.23930613]])], [array([[[[12148.65451011],
           [17211.56717148],
           [18415.60424182]]],
  
  
         [[[17211.56717148],
           [25168.80037002],
           [26476.97723468]]],
  
  
         [[[18415.60424182],
           [26476.97723468],
           [28287.02854993]]]])]]

In [7]:
x[0,0]

<tf.Tensor 'strided_slice:0' shape=() dtype=float64>

### 2nd order derivative

Our model starts with a vector valued input $\vec{x}$, maps it to some intermediate value $\vec{a}$ and finally produces a scalar y. The information flow looks like this:
$$
\begin{bmatrix}x_1 \\ x_2 \\ \vdots \\x_n \end{bmatrix} \to 
\begin{bmatrix}a_1 \\ a_2 \\ \vdots \\a_n \end{bmatrix} \to 
y
$$

We are interested in the first and second derivatives of y with respect to x. Since y is a scalar, the Jacobian matrix is a row vector, and the second derivative can be expressed as a Hessian: 
$$
J_{yx}, (J_{yx})_i= \frac{\partial y}{\partial x_i}\\
H_{yx}, (H_{yx})_{ij} = \frac{\partial^2y}{\partial x_i \partial x_j}
$$

The first and second derivative of y with respect to a are given. The naming scheme is the same, we use $J_{ya}$ and $H_{ya}$

Now we want to use this to calculate $J_{yx}$ and $H_{yx}$.

The Jacobian of y with respect to x is straightforward. First we calculate the Jacobian $J_{ax}$, since a is a vector, this is a matrix. To get the Jacobian $J_{yx}$, we use a matrix product:
$$
(J_{ax})_{ij} = \frac{\partial a_i}{\partial x_j} \\
(J_{yx})_{j} = \sum_i (J_{ya})_i * (J_{ax})_{ij}
$$

The Hessian of y with respect to x is complicated. We need to look at all the paths through a, to compute the derivatives:
$$
\begin{align}
(H_{yx})_{ij}&= \frac{\partial^2 y}{\partial x_i \partial x_j} \\
&= \sum_{h,k} \frac{\partial^2 y}{\partial a_h \partial a_k} \frac{\partial a_h}{\partial x_i} \frac{\partial a_k}{\partial x_j} \\
&\; +\sum_h \frac{\partial y}{\partial a_h} \frac{\partial^2 a_h}{\partial x_i \partial x_j}
\end{align}
$$

Apparently the Hessian consists of two components:
$$
\begin{align}
(H_{yx})_{ij} &= (I_{yx})_{ij} + (O_{yx})_{ij} \\
(I_{yx})_{ij} &= \sum_{h,k} \frac{\partial^2 y}{\partial a_h \partial a_k} \frac{\partial a_h}{\partial x_i} \frac{\partial a_k}{\partial x_j} \\
(O_{yx})_{ij} &=\sum_h \frac{\partial y}{\partial a_h} \frac{\partial^2 a_h}{\partial x_i \partial x_j}
\end{align}
$$

We can rewrite this in terms of the given Jacobians and Hessians:
$$
\begin{align}
(I_{yx})_{i,j} &= \sum_{h,k} (H_{ya})_{h,k} (J_{ax})_{h,i} (J_{ax})_{k,j} \\
(O_{yx})_{i,j} &= \sum_{h} (J_{ya})_{1, h} (H_{ax})_{h,i,j}
\end{align}
$$

Please note the Hessian $H_{ax}$, this is indexed with 3 indices. Because a is a vector and we take the second derivative with respect to a vector, this becomes a 3-dimensional tensor. To avoid confusion, I'm using two indices for $(J_{ya})_{1, h}$, this is a row vector.

### coding samples
Now I will look at concerete implementations. To stay compatible with the notation above, I assume that our layer maps x to a and that the whole model finally outputs y.

In [8]:
import numpy as np

In [9]:
class LinearLayer():
    def __init__(W, b):
        self.W = W
        self.b = b
        
        self.J_y={}
        self.H_y={}
        
    def forward(x):
        self.x = x
        self.a = W@x + b
        return self.a
    
    def backward(J_ya):
        self.J_ya = J_ya

        # update the first derivatives of our params
        self.J_y["W"][:] = np.dot(dy_dout, self.inp.T)
        self.J_y["b"][:] = np.sum(dy_dout, keepdims=True, axis=1)

        self.J_ax = self.W.T
        self.J_yx = J_ya @ self.J_ax

        return self.J_yx